## Sitting Posture Detection Training

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import f1_score, recall_score, accuracy_score
import os
import datetime
import itertools
import csv

### Define Dataset class

In [12]:
class PostureDataset(Dataset):
    def __init__(self, data, scaler=None, label_encoder=None, from_csv=True):
        """
        Args:
        - data: CSV file path or DataFrame containing the dataset.
        - scaler: StandardScaler for feature normalization.
        - label_encoder: LabelEncoder for encoding the labels (not required if already encoded).
        - from_csv: Set to True if the input is a CSV path.
        """
        self.label_encoder = label_encoder

        # Load data
        if from_csv:
            data = pd.read_csv(data)
        
        # Ensure required columns are present
        if 'class' not in data.columns:
            raise ValueError("The input data must contain a 'class' column.")
        
        # Extract features and labels
        self.X = data.drop(columns=['class']).values  # Features
        self.y = data['class'].values  # Labels (already encoded)

        # Normalize features using StandardScaler
        if scaler:
            self.X = scaler.transform(self.X)
        else:
            self.scaler = StandardScaler()
            self.X = self.scaler.fit_transform(self.X)
        
        self.X = torch.tensor(self.X, dtype=torch.float32)
        self.y = torch.tensor(self.y, dtype=torch.long)  # Ensure labels are integers

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

### Define ANN Model Architecture

In [13]:
class MLP(nn.Module):
    def __init__(self, input_size, num_classes, num_hidden_layers, neurons_per_layer, dropout_rate):
        super(MLP, self).__init__()
        
        # Generate a decreasing neuron configuration based on the initial size
        # For example, if neurons_per_layer=512 and num_hidden_layers=3, 
        # we'll create [512, 256, 128]
        if not isinstance(neurons_per_layer, (list, tuple)):
            initial_neurons = neurons_per_layer
            neurons_list = []
            for i in range(num_hidden_layers):
                neurons_list.append(initial_neurons // (2**i))
        else:
            neurons_list = neurons_per_layer
            
        # Create the layers list
        layers = []
        
        # First hidden layer (from input_size to first layer size)
        layers.append(nn.Linear(input_size, neurons_list[0]))
        layers.append(nn.BatchNorm1d(neurons_list[0]))
        layers.append(nn.LeakyReLU(negative_slope=0.01))
        layers.append(nn.Dropout(dropout_rate))
        
        # Additional hidden layers with different sizes
        for i in range(1, num_hidden_layers):
            layers.append(nn.Linear(neurons_list[i-1], neurons_list[i]))
            layers.append(nn.BatchNorm1d(neurons_list[i]))
            layers.append(nn.LeakyReLU(negative_slope=0.01))
            layers.append(nn.Dropout(dropout_rate))
        
        # Output layer
        layers.append(nn.Linear(neurons_list[-1], num_classes))
        
        # Create the sequential model
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

### Define the Training & Evaluation Function

In [14]:
def evaluate_model(model, data_loader, device):
    """
    Evaluates the model and returns accuracy, recall, and F1 score.
    """
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    # Calculate metrics
    accuracy = accuracy_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')
    
    return accuracy * 100, recall * 100, f1 * 100

def train_model(model, train_loader, val_loader, criterion, optimizer, epochs, patience=15):
    """
    Train the model with early stopping based on validation loss.
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    # Initialize lists to store metrics
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []
    
    # Early stopping variables
    best_val_loss = float('inf')
    early_stop_counter = 0
    
    for epoch in range(epochs):
        # Training Phase
        model.train()
        train_loss = 0
        correct_train, total_train = 0, 0
        
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()
        
        train_acc = correct_train / total_train
        train_losses.append(train_loss / len(train_loader))
        train_accuracies.append(train_acc)
        
        # Validation Phase
        model.eval()
        val_loss, correct_val, total_val = 0, 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()
        
        val_acc = correct_val / total_val
        current_val_loss = val_loss / len(val_loader)
        val_losses.append(current_val_loss)
        val_accuracies.append(val_acc)
        
        # Early stopping check
        if current_val_loss < best_val_loss:
            best_val_loss = current_val_loss
            early_stop_counter = 0
        else:
            early_stop_counter += 1
            if early_stop_counter >= patience:
                break
    
    return train_losses, train_accuracies, val_losses, val_accuracies

### Defining Dataset Loading Function

In [15]:
# Load and preprocess data
def load_and_prepare_data(csv_path):
    """
    Load the dataset, encode labels, and prepare for training.
    """
    df = pd.read_csv(csv_path)  
    
    print("Number of samples per class before train test split:")
    print(df['class'].value_counts())
    
    # Encode class labels to integers using LabelEncoder
    label_encoder = LabelEncoder()
    df['class'] = label_encoder.fit_transform(df['class'])
    
    # Split the data into training and test datasets
    RANDOM_SEED = 42
    torch.manual_seed(RANDOM_SEED)
    torch.cuda.manual_seed(RANDOM_SEED)
    
    train_data, test_data = train_test_split(df, test_size=0.3, random_state=RANDOM_SEED)
    
    # Standardize features
    scaler = StandardScaler()
    train_features = train_data.drop(columns=['class']).values
    scaler.fit(train_features)
    
    return train_data, test_data, scaler, label_encoder

### Function to run single experiment

In [16]:
def run_experiment(hyperparams, train_data, test_data, scaler, label_encoder, exp_id, num_classes=4):
    """
    Run a single experiment with the given hyperparameters and return the results.
    """
    # Extract hyperparameters
    learning_rate = hyperparams['learning_rate']
    num_hidden_layers = hyperparams['num_hidden_layers']
    neurons_per_layer = hyperparams['neurons_per_layer']
    dropout_rate = hyperparams['dropout_rate']
    batch_size = hyperparams['batch_size']
    epochs = hyperparams['epochs']
    
    # Create datasets and loaders
    train_dataset = PostureDataset(train_data, scaler=scaler, label_encoder=label_encoder, from_csv=False)
    test_dataset = PostureDataset(test_data, scaler=scaler, label_encoder=label_encoder, from_csv=False)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    # Model initialization
    input_size = train_data.drop(columns=['class']).shape[1]
    
    model = MLP(input_size, num_classes, num_hidden_layers, neurons_per_layer, dropout_rate)
    
    # Loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0.005)
    
    # Train the model
    train_losses, train_accuracies, val_losses, val_accuracies = train_model(
        model, train_loader, test_loader, criterion, optimizer, epochs
    )
    
    # Evaluate the model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    accuracy, recall, f1 = evaluate_model(model, test_loader, device)
    
    # Store metrics
    metrics = {
        'accuracy': accuracy,
        'recall': recall,
        'f1_score': f1,
        'best_val_loss': min(val_losses),
        'best_val_accuracy': max(val_accuracies)
    }
    
    return metrics

### Defining the Grid Search Function

In [ ]:
def hyperparameter_grid_search(grid, train_data, test_data, scaler, label_encoder, csv_file='../experiments/tuning/tuning_results.csv'):
    """
    Perform a grid search over the hyperparameter space and save only the CSV results.
    """
    # Create directory for the CSV if needed
    os.makedirs(os.path.dirname(os.path.abspath(csv_file)) if os.path.dirname(csv_file) else '.', exist_ok=True)
    
    # Define CSV headers
    headers = [
        'Experiment', 'Number of Layers', 'Neurons per Layer', 'Learning Rate',
        'Dropout Rate', 'Batch Size', 'Epochs', 'Optimizer', 'Accuracy', 'Recall', 'Value F1'
    ]
    
    # Write the header row with explicit UTF-8 encoding
    with open(csv_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(headers)
    
    # Generate all combinations of hyperparameters
    keys = grid.keys()
    values = grid.values()
    combinations = list(itertools.product(*values))
    total_experiments = len(combinations)
    
    # Run each experiment
    results = []
    
    print(f"Starting grid search with {total_experiments} experiments...")
    
    for i, combo in enumerate(combinations):
        # Create hyperparameter dictionary
        hyperparams = dict(zip(keys, combo))
        
        # Create experiment ID
        exp_id = f"Exp-{i+1}"
        
        # Generate the decreasing neuron configuration
        initial_neurons = hyperparams['neurons_per_layer']

        # Add this check to handle if initial_neurons is already a list
        if isinstance(initial_neurons, (list, tuple)):
            neuron_config = initial_neurons  # Use it directly
        else:
            # Otherwise, generate decreasing neuron pattern
            num_layers = hyperparams['num_hidden_layers']
            neuron_config = []
            for j in range(num_layers):
                neuron_config.append(initial_neurons // (2**j))
        
        # Store the complete neuron configuration
        hyperparams['neuron_config'] = neuron_config
        
        # Print progress info
        print(f"Running {exp_id} of {total_experiments} ({(i+1)/total_experiments*100:.1f}%)")
        
        # Run the experiment
        metrics = run_experiment(
            hyperparams, train_data, test_data, 
            scaler, label_encoder, exp_id
        )
        
        # Prepare row for CSV
        row = [
            exp_id,
            hyperparams['num_hidden_layers'],
            str(neuron_config),  # Store as string representation of list
            hyperparams['learning_rate'],
            hyperparams['dropout_rate'],
            hyperparams['batch_size'],
            hyperparams['epochs'],
            'Adam',  # Fixed to Adam
            f"{metrics['accuracy']:.2f}",
            f"{metrics['recall']:.2f}",
            f"{metrics['f1_score']:.2f}"
        ]
        
        # Write the results to the CSV file with explicit UTF-8 encoding
        with open(csv_file, 'a', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(row)
        
        # Store results
        results.append({
            'hyperparams': hyperparams,
            'metrics': metrics
        })
        
        # Print results for this experiment
        print(f"Results for {exp_id}: Accuracy={metrics['accuracy']:.2f}%, Recall={metrics['recall']:.2f}%, F1={metrics['f1_score']:.2f}%")
        print("-" * 40)
    
    return results, csv_file

### Function to visualise hyperparameter results

In [ ]:
def visualize_hyperparameter_results(csv_file, output_dir='../experiments/tuning'):
    """
    Visualize the hyperparameter tuning results in a more compact way.
    Creates heatmaps and bar charts to display the results.
    """
    import matplotlib.pyplot as plt
    import seaborn as sns
    import pandas as pd
    import os
    import numpy as np
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Load results
    results = pd.read_csv(csv_file)
    
    # Convert string metrics to numeric
    for col in ['Accuracy', 'Recall', 'Value F1']:
        results[col] = pd.to_numeric(results[col])
    
    # 1. Top Performers Table - show only top 15 results sorted by accuracy
    top_performers = results.sort_values('Accuracy', ascending=False).head(15)
    print("Top 15 Configurations by Accuracy:")
    print(top_performers)
    
    # Save top performers to CSV
    top_performers.to_csv(os.path.join(output_dir, 'top_performers.csv'), index=False)
    
    # ---------- VISUALIZATIONS ----------
    
    # 2. Create plots in a 2x3 grid
    plt.figure(figsize=(20, 15))
    
    # 2.1 Learning Rate vs Dropout Rate (with accuracy as color)
    plt.subplot(2, 3, 1)
    pivot = results.pivot_table(
        values='Accuracy', 
        index='Learning Rate',
        columns='Dropout Rate', 
        aggfunc='mean'
    )
    sns.heatmap(pivot, annot=True, cmap='YlGnBu', fmt='.1f')
    plt.title('Mean Accuracy by Learning Rate and Dropout Rate')
    
    # 2.2 Learning Rate vs Number of Layers
    plt.subplot(2, 3, 2)
    pivot = results.pivot_table(
        values='Accuracy', 
        index='Learning Rate',
        columns='Number of Layers', 
        aggfunc='mean'
    )
    sns.heatmap(pivot, annot=True, cmap='YlGnBu', fmt='.1f')
    plt.title('Mean Accuracy by Learning Rate and Network Depth')
    
    # 2.3 Epochs vs Number of Layers
    plt.subplot(2, 3, 3)
    pivot = results.pivot_table(
        values='Accuracy', 
        index='Epochs',
        columns='Number of Layers', 
        aggfunc='mean'
    )
    sns.heatmap(pivot, annot=True, cmap='YlGnBu', fmt='.1f')
    plt.title('Mean Accuracy by Epochs and Network Depth')
    
    # 2.4 Bar chart of metrics by Number of Layers
    plt.subplot(2, 3, 4)
    metrics_by_layers = results.groupby('Number of Layers')[['Accuracy', 'Recall', 'Value F1']].mean().reset_index()
    metrics_by_layers.plot(x='Number of Layers', kind='bar', ax=plt.gca())
    plt.title('Performance Metrics by Network Depth')
    plt.ylabel('Score')
    plt.xticks(rotation=0)
    
    # 2.5 Bar chart of metrics by Learning Rate
    plt.subplot(2, 3, 5)
    metrics_by_lr = results.groupby('Learning Rate')[['Accuracy', 'Recall', 'Value F1']].mean().reset_index()
    metrics_by_lr.plot(x='Learning Rate', kind='bar', ax=plt.gca())
    plt.title('Performance Metrics by Learning Rate')
    plt.ylabel('Score')
    plt.xticks(rotation=0)
    
    # 2.6 Bar chart of metrics by Epochs
    plt.subplot(2, 3, 6)
    metrics_by_epochs = results.groupby('Epochs')[['Accuracy', 'Recall', 'Value F1']].mean().reset_index()
    metrics_by_epochs.plot(x='Epochs', kind='bar', ax=plt.gca())
    plt.title('Performance Metrics by Epochs')
    plt.ylabel('Score')
    plt.xticks(rotation=0)
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'hyperparameter_visualization_grid.png'))
    plt.close()
    
    # 3. Network architectures comparison
    plt.figure(figsize=(14, 8))
    
    # Create a more readable representation of network architecture
    results['Network Architecture'] = results.apply(
        lambda row: f"{row['Number of Layers']}L: {row['Neurons per Layer']}", axis=1
    )
    
    # Get top 2 performing architectures for each depth
    top_archs = []
    for depth in results['Number of Layers'].unique():
        filtered = results[results['Number of Layers'] == depth]
        best_archs = filtered.groupby('Neurons per Layer')['Accuracy'].mean().nlargest(1).index.tolist()
        for arch in best_archs:
            top_archs.append((depth, arch))
    
    # Filter results to only include top architectures
    filtered_results = results[results.apply(
        lambda row: (row['Number of Layers'], row['Neurons per Layer']) in top_archs, axis=1
    )]
    
    # Get mean metrics for each architecture
    arch_metrics = filtered_results.groupby('Network Architecture')[['Accuracy', 'Recall', 'Value F1']].mean()
    
    # Sort by accuracy
    arch_metrics = arch_metrics.sort_values('Accuracy', ascending=False)
    
    # Plot
    arch_metrics.plot(kind='barh', figsize=(10, 8))
    plt.title('Performance Comparison of Top Network Architectures')
    plt.xlabel('Score')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'network_architecture_comparison.png'))
    plt.close()
    
    # 4. Learning curves - Accuracy vs Epochs for different configurations
    plt.figure(figsize=(14, 8))
    
    # Group by epochs and compute mean, min, max
    epoch_stats = results.groupby('Epochs')[['Accuracy', 'Recall', 'Value F1']].agg(['mean', 'min', 'max'])
    
    # Plot accuracy vs epochs with error bars
    x = np.array(epoch_stats.index)
    y = epoch_stats['Accuracy']['mean'].values
    min_vals = epoch_stats['Accuracy']['min'].values
    max_vals = epoch_stats['Accuracy']['max'].values
    
    plt.errorbar(x, y, yerr=[y-min_vals, max_vals-y], fmt='-o', capsize=5, label='Accuracy')
    
    # Plot recall and F1 as well
    plt.plot(x, epoch_stats['Recall']['mean'].values, 's-', label='Recall')
    plt.plot(x, epoch_stats['Value F1']['mean'].values, '^-', label='F1 Score')
    
    plt.xlabel('Epochs')
    plt.ylabel('Score')
    plt.title('Performance Metrics vs Training Epochs')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.savefig(os.path.join(output_dir, 'epoch_performance.png'))
    plt.close()
    
    # 5. Parameter importance analysis
    plt.figure(figsize=(12, 8))
    
    # Compute variance in accuracy for each parameter
    param_variance = {}
    for param in ['Learning Rate', 'Number of Layers', 'Dropout Rate', 'Epochs']:
        param_variance[param] = results.groupby(param)['Accuracy'].mean().var()
    
    # Convert to DataFrame and sort
    param_var_df = pd.DataFrame.from_dict(param_variance, orient='index', columns=['Variance'])
    param_var_df = param_var_df.sort_values('Variance', ascending=False)
    
    # Plot
    param_var_df.plot(kind='bar', figsize=(10, 6))
    plt.title('Hyperparameter Importance (Higher Variance = More Impact)')
    plt.ylabel('Variance in Accuracy')
    plt.xlabel('Hyperparameter')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'parameter_importance.png'))
    plt.close()
    
    # 6. Create a table-like visualization summarizing the best configurations
    plt.figure(figsize=(12, 8))
    
    # Get top configurations for each metric
    top_configs = {
        'Accuracy': results.sort_values('Accuracy', ascending=False).iloc[0],
        'Recall': results.sort_values('Recall', ascending=False).iloc[0],
        'F1 Score': results.sort_values('Value F1', ascending=False).iloc[0]
    }
    
    # Create table data
    table_data = [
        [metric, 
         f"{config['Learning Rate']}", 
         f"{config['Number of Layers']}", 
         f"{config['Neurons per Layer']}", 
         f"{config['Dropout Rate']}", 
         f"{config['Epochs']}", 
         f"{config['Accuracy']:.2f}", 
         f"{config['Recall']:.2f}", 
         f"{config['Value F1']:.2f}"]
        for metric, config in top_configs.items()
    ]
    
    # Create table
    ax = plt.gca()
    ax.axis('off')
    table = ax.table(
        cellText=table_data,
        colLabels=['Optimized For', 'LR', 'Layers', 'Neurons', 'Dropout', 'Epochs', 'Accuracy', 'Recall', 'F1'],
        loc='center',
        cellLoc='center'
    )
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1.2, 1.5)
    
    plt.title('Best Configurations by Metric', y=0.8)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'best_configs_table.png'))
    plt.close()
    
    print(f"Visualizations saved to {output_dir}")
    
    # Return detailed statistics for further analysis
    return {
        'top_by_accuracy': results.sort_values('Accuracy', ascending=False).head(5),
        'top_by_f1': results.sort_values('Value F1', ascending=False).head(5),
        'top_by_recall': results.sort_values('Recall', ascending=False).head(5),
        'mean_by_lr': results.groupby('Learning Rate')['Accuracy'].mean(),
        'mean_by_layers': results.groupby('Number of Layers')['Accuracy'].mean(),
        'mean_by_dropout': results.groupby('Dropout Rate')['Accuracy'].mean(),
        'mean_by_epochs': results.groupby('Epochs')['Accuracy'].mean(),
    }

### Main Execution Function

In [19]:
# Main execution function
def main():
    # Define the hyperparameter grid
    hyperparameter_grid = {
        'learning_rate': [0.001, 0.0005, 0.0001],
        'num_hidden_layers': [2, 3, 4],
        'neurons_per_layer': [128, 256, 512],
        'dropout_rate': [0.3, 0.5, 0.7],
        'batch_size': [16, 32],
        'epochs': [100, 200, 300]
    }

    # Alternative simplified grid for faster testing
    simplified_grid = {
        'learning_rate': [0.001],
        'num_hidden_layers': [3],
        'neurons_per_layer': [[512, 256, 128]],
        'dropout_rate': [0.4],
        'batch_size': [32],
        'epochs': [100],
        'optimizer': ['Adam']
    }

    # Load and prepare data
    csv_path = "../../datasets/vectors/augmented_xy_filtered_keypoints_vectors_mediapipe.csv"
    train_data, test_data, scaler, label_encoder = load_and_prepare_data(
        csv_path)

    # Run grid search with simplified grid (for testing)
    # results, csv_file = hyperparameter_grid_search(simplified_grid, train_data, test_data, scaler, label_encoder)

    # For the full grid, uncomment this line
    results, csv_file = hyperparameter_grid_search(
        hyperparameter_grid, train_data, test_data, scaler, label_encoder)

    print(f"All experiments completed. Results saved to {csv_file}")

    # Load the results as a DataFrame and display
    df_results = pd.read_csv(csv_file)
    print(df_results)
    
    # Visualize results
    print("\nCreating visualizations...")
    stats = visualize_hyperparameter_results(csv_file)
    
    print("\nTop 5 configurations by accuracy:")
    print(stats['top_by_accuracy'])
    
    print("\nMean accuracy by parameter:")
    print("Learning Rate:", stats['mean_by_lr'])
    print("Network Depth:", stats['mean_by_layers'])
    print("Dropout Rate:", stats['mean_by_dropout'])
    print("Epochs:", stats['mean_by_epochs'])


if __name__ == "__main__":
    main()

Number of samples per class before train test split:
class
crossed_legs    1091
proper          1084
slouching       1000
reclining        970
Name: count, dtype: int64
Starting grid search with 486 experiments...
Running Exp-1 of 486 (0.2%)
Results for Exp-1: Accuracy=81.43%, Recall=81.29%, F1=81.38%
----------------------------------------
Running Exp-2 of 486 (0.4%)
Results for Exp-2: Accuracy=81.75%, Recall=81.41%, F1=81.59%
----------------------------------------
Running Exp-3 of 486 (0.6%)
Results for Exp-3: Accuracy=81.75%, Recall=81.58%, F1=81.56%
----------------------------------------
Running Exp-4 of 486 (0.8%)
Results for Exp-4: Accuracy=84.57%, Recall=84.54%, F1=84.42%
----------------------------------------
Running Exp-5 of 486 (1.0%)
Results for Exp-5: Accuracy=84.32%, Recall=84.11%, F1=84.18%
----------------------------------------
Running Exp-6 of 486 (1.2%)
Results for Exp-6: Accuracy=84.16%, Recall=84.02%, F1=84.02%
----------------------------------------
Runnin

<Figure size 1400x800 with 0 Axes>

<Figure size 1200x800 with 0 Axes>